<a href="https://colab.research.google.com/github/sheetal1996/Eco-lution/blob/main/575DataCleanign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import random
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [1]:
#https://www.geeksforgeeks.org/ways-to-import-csv-files-in-google-colab/
from google.colab import files
 
 
uploaded = files.upload()

Saving owid-co2-data.csv to owid-co2-data.csv


In [3]:
import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['owid-co2-data.csv']))
print(df. columns) 

Index(['iso_code', 'country', 'year', 'co2', 'co2_per_capita', 'trade_co2',
       'cement_co2', 'cement_co2_per_capita', 'coal_co2',
       'coal_co2_per_capita', 'flaring_co2', 'flaring_co2_per_capita',
       'gas_co2', 'gas_co2_per_capita', 'oil_co2', 'oil_co2_per_capita',
       'other_industry_co2', 'other_co2_per_capita', 'co2_growth_prct',
       'co2_growth_abs', 'co2_per_gdp', 'co2_per_unit_energy',
       'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_co2', 'cumulative_cement_co2',
       'cumulative_coal_co2', 'cumulative_flaring_co2', 'cumulative_gas_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'trade_co2_share',
       'share_global_co2', 'share_global_cement_co2', 'share_global_coal_co2',
       'share_global_flaring_co2', 'share_global_gas_co2',
       'share_global_oil_co2', 'share_global_other_co2',
       'share_global_cumulative_co2', 'share_global_cumulative_cement_co2',
       'share_global_cumulative_c

In [ ]:
df.shape

In [5]:
#Let us first find out the columns which have atleast 50% of non-null row
df.columns[df.notnull().mean() >= 0.5]



Index(['iso_code', 'country', 'year', 'co2', 'co2_per_capita', 'coal_co2',
       'coal_co2_per_capita', 'oil_co2', 'oil_co2_per_capita',
       'co2_growth_prct', 'co2_growth_abs', 'co2_per_gdp', 'cumulative_co2',
       'cumulative_coal_co2', 'cumulative_oil_co2', 'share_global_co2',
       'share_global_coal_co2', 'share_global_oil_co2',
       'share_global_cumulative_co2', 'share_global_cumulative_coal_co2',
       'share_global_cumulative_oil_co2', 'population', 'gdp'],
      dtype='object')

In [22]:
#Since we need to remove the other columns with less than 50% of non-null rows, we use the 'drop' function
#we assign the resulting dataframe to a new variable name 'df_dropcol'
df_dropcol = df.drop(df.columns[df.notnull().mean() < 0.5],axis=1)

#Now, if we see again the information about this new df_dropcol dataframe (by using the info function), 
#we can see that we still have null values. So, in order to remove these null values, we can use dropna() function on df_dropcol.

df_dropcol.dropna(inplace=True)
df_dropcol.shape

(8880, 23)

> Step 1: We have to find a way to fill the null values.
If we are doing this, we are losing 2020 data as well. Even if we are directly removing the null values using dropna(), we are losing even higher data.


In [18]:
df_dropcol

,iso_code,country,year,co2,co2_per_capita,coal_co2,coal_co2_per_capita,oil_co2,oil_co2_per_capita,co2_growth_prct,...,cumulative_coal_co2,cumulative_oil_co2,share_global_co2,share_global_coal_co2,share_global_oil_co2,share_global_cumulative_co2,share_global_cumulative_coal_co2,share_global_cumulative_oil_co2,population,gdp
1,AFG,Afghanistan,1950,0.084,0.011,0.021,0.003,0.063,0.008,475.00,...,0.036,0.063,0.00,0.00,0.00,0.00,0.00,0.00,7752117.0,9.421400e+09
2,AFG,Afghanistan,1951,0.092,0.012,0.026,0.003,0.066,0.008,8.70,...,0.061,0.129,0.00,0.00,0.00,0.00,0.00,0.00,7840151.0,9.692280e+09
3,AFG,Afghanistan,1952,0.092,0.012,0.032,0.004,0.060,0.008,0.00,...,0.093,0.189,0.00,0.00,0.00,0.00,0.00,0.00,7935996.0,1.001733e+10
4,AFG,Afghanistan,1953,0.106,0.013,0.038,0.005,0.068,0.008,16.00,...,0.131,0.257,0.00,0.00,0.00,0.00,0.00,0.00,8039684.0,1.063052e+10
5,AFG,Afghanistan,1954,0.106,0.013,0.043,0.005,0.064,0.008,0.00,...,0.174,0.321,0.00,0.00,0.00,0.00,0.00,0.00,8151316.0,1.086636e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25982,ZWE,Zimbabwe,2014,11.906,0.876,7.691,0.566,3.719,0.274,0.59,...,570.112,126.428,0.03,0.05,0.03,0.05,0.08,0.02,13586710.0,2.122250e+10
25983,ZWE,Zimbabwe,2015,12.226,0.885,8.033,0.582,3.608,0.261,2.69,...,578.145,130.036,0.03,0.05,0.03,0.05,0.08,0.02,13814642.0,2.102745e+10
25984,ZWE,Zimbabwe,2016,10.738,0.765,6.959,0.496,3.139,0.224,-12.17,...,585.104,133.175,0.03,0.05,0.03,0.05,0.08,0.02,14030338.0,2.096179e+10
25985,ZWE,Zimbabwe,2017,9.582,0.673,5.665,0.398,3.239,0.228,-10.77,...,590.768,136.414,0.03,0.04,0.03,0.05,0.08,0.02,14236599.0,2.194784e+10


In [17]:
df_gdp_2018 = df_dropcol.loc[df_dropcol['year']==2020] # finds all the countries with data based on 201

#df_gdp_top10 = df_gdp_2018.sort_values(by=['gdp'], ascending = False).head(10) #arranges the countries based on their GDP in 2020

#print(df_gdp_2018['country']) # prints the top 10 countries with highest GDP8
print(df_gdp_2018.to_markdown())

| iso_code   | country   | year   | co2   | co2_per_capita   | coal_co2   | coal_co2_per_capita   | oil_co2   | oil_co2_per_capita   | co2_growth_prct   | co2_growth_abs   | co2_per_gdp   | cumulative_co2   | cumulative_coal_co2   | cumulative_oil_co2   | share_global_co2   | share_global_coal_co2   | share_global_oil_co2   | share_global_cumulative_co2   | share_global_cumulative_coal_co2   | share_global_cumulative_oil_co2   | population   | gdp   |
|------------|-----------|--------|-------|------------------|------------|-----------------------|-----------|----------------------|-------------------|------------------|---------------|------------------|-----------------------|----------------------|--------------------|-------------------------|------------------------|-------------------------------|------------------------------------|-----------------------------------|--------------|-------|


In [27]:
df_nona = df.dropna()

#Investigating the shape of 'df_nona'

df_nona.shape

#df_nona['year'].max() # This line of code will return the most recent year in the data set

df_dropcol['year'].max()

2018

In [29]:
#Selecting some columns:
selected_columns = ["country","year","co2","consumption_co2","trade_co2","cement_co2","coal_co2","gas_co2","oil_co2","other_industry_co2","co2_per_capita","population"]

co2_df = df[selected_columns] # Filtering out the selected columns

co2_df

,country,year,co2,consumption_co2,trade_co2,cement_co2,coal_co2,gas_co2,oil_co2,other_industry_co2,co2_per_capita,population
0,Afghanistan,1949,0.015,NaN,NaN,NaN,0.015,NaN,NaN,NaN,0.002,7624058.0
1,Afghanistan,1950,0.084,NaN,NaN,NaN,0.021,NaN,0.063,NaN,0.011,7752117.0
2,Afghanistan,1951,0.092,NaN,NaN,NaN,0.026,NaN,0.066,NaN,0.012,7840151.0
3,Afghanistan,1952,0.092,NaN,NaN,NaN,0.032,NaN,0.060,NaN,0.012,7935996.0
4,Afghanistan,1953,0.106,NaN,NaN,NaN,0.038,NaN,0.068,NaN,0.013,8039684.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25984,Zimbabwe,2016,10.738,12.153,1.415,0.639,6.959,NaN,3.139,NaN,0.765,14030338.0
25985,Zimbabwe,2017,9.582,11.248,1.666,0.678,5.665,NaN,3.239,NaN,0.673,14236599.0
25986,Zimbabwe,2018,11.854,13.163,1.308,0.697,7.101,NaN,4.056,NaN,0.821,14438812.0
25987,Zimbabwe,2019,10.949,12.422,1.473,0.697,6.020,NaN,4.232,NaN,0.748,14645473.0


In [30]:
#aggregating the data by years:

co2_df.groupby(by = "country").sum()

,year,co2,consumption_co2,trade_co2,cement_co2,coal_co2,gas_co2,oil_co2,other_industry_co2,co2_per_capita,population
country,,,,,,,,,,,
Afghanistan,142884,192.851,0.000,0.000,2.505,60.410,20.152,103.834,0.000,9.525,1.230725e+09
Africa,267424,47610.749,22120.671,-8892.489,1862.329,20304.715,5948.868,16708.257,0.000,110.011,5.493399e+10
Albania,173932,285.504,143.572,29.578,19.479,66.963,16.602,182.455,0.000,111.049,1.997508e+08
Algeria,206640,4587.133,0.000,0.000,229.285,116.452,2062.655,1590.075,0.000,162.607,1.893386e+09
Andorra,62155,15.072,0.000,0.000,0.000,0.000,0.000,15.072,0.000,209.122,2.252151e+06
...,...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,40210,0.508,0.000,0.000,0.000,0.000,0.000,0.508,0.000,38.943,2.631380e+05
World,510835,1696524.172,1407952.483,0.000,43163.195,788362.041,245231.882,592621.162,7725.986,400.301,5.852030e+11
Yemen,140935,624.651,0.000,0.000,22.830,4.712,14.052,583.064,0.000,45.468,9.037078e+08


In [31]:
#Top 30 countries of the world by emission
co2_top = co2_df.groupby(by = "country").sum()["co2"]
co2_top = co2_top.sort_values(ascending=False) 
co2_top.head(30)

country
World                            1696524.172
High-income countries             966097.317
Asia                              532916.151
Europe                            531155.787
Upper-middle-income countries     504295.273
North America                     477801.969
United States                     416723.088
European Union (28)               368286.501
European Union (27)               290125.368
Asia (excl. China & India)        242935.659
Europe (excl. EU-27)              241030.431
China                             235564.016
Lower-middle-income countries     165469.908
Europe (excl. EU-28)              162869.282
Russia                            115344.148
Germany                            92635.618
United Kingdom                     78161.148
Japan                              65625.165
North America (excl. USA)          61078.901
India                              54416.494
Africa                             47610.749
South America                      43288.717
In

In [34]:
#A way to fill the data using average:
co2_asia = co2_df[co2_df["country"] == "Asia"] # Filerting out Asian data
co2_europe = co2_df[co2_df["country"] == "Europe"]

co2_asia_null = co2_asia[co2_asia["co2"].isnull()][["year","co2"]] # Checking for NULL values
co2_asia_null


,year,co2
1057,1834,NaN
1058,1835,NaN
1059,1836,NaN
1060,1837,NaN
1061,1838,NaN
1062,1839,NaN
1063,1840,NaN
1064,1841,NaN
1065,1842,NaN
1066,1843,NaN


In [36]:

for i in co2_asia_null.index:
  co2_asia.at[i, "co2"] = co2_asia.at[1056, "co2"] + co2_asia.at[1081, "co2"] # Index of the years 1833 and 1857 are 1056 and 1081 respectively
co2_asia.loc[1056:1081][["country", "year", "co2"]]

,country,year,co2
1056,Asia,1833,0.002
1057,Asia,1834,0.460
1058,Asia,1835,0.460
1059,Asia,1836,0.460
1060,Asia,1837,0.460
1061,Asia,1838,0.460
1062,Asia,1839,0.460
1063,Asia,1840,0.460
1064,Asia,1841,0.460
1065,Asia,1842,0.460
